# RF6 Impact Attestation Metrics

Spec [here](https://plaid-cement-e44.notion.site/Impact-Attestations-Data-b8b1c79a624c46ad94305c74def64783).

In [1]:
import datetime
import json
import pandas as pd

from scripts.agora import agora_api
from scripts.farcaster import load_farcaster
from scripts.metrics import calculate_metrics

# Load data

In [2]:
f = load_farcaster()
FARCASTER = pd.DataFrame(f).groupby('fid')['address'].apply(set).to_dict()

In [3]:
delegate_data = agora_api('delegates', params={'limit': 100, 'offset': 0, 'sort': 'most_delegators'})
rf6_projects = agora_api('retrofunding/rounds/6/projects')

DELEGATES = [x['address'].lower() for x in delegate_data]
AGORA_IDS = [x['projectId'] for x in rf6_projects]

Fetched a total of 100 items from delegates.
Fetched 100 items, offset now at 100
Fetched a total of 142 items from retrofunding/rounds/6/projects.


In [4]:
def load_json(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    return data

project_attestation_data = load_json('data/attestations/472.json')
print("Project attestations:", len(project_attestation_data))

app_attestation_data = load_json('data/attestations/609.json')
print("Application attestations:", len(app_attestation_data))

badgeholder_attestation_data = load_json('data/attestations/599.json')
print("Badgeholder attestations:", len(badgeholder_attestation_data))

gov_council_attestation_data = load_json('data/attestations/141.json')
print("Gov Council attestations:", len(gov_council_attestation_data))

impact_attestation_data = load_json('data/attestations/566.json')
print("Impact attestations:", len(impact_attestation_data))

Project attestations: 4267
Application attestations: 407
Badgeholder attestations: 305
Gov Council attestations: 217
Impact attestations: 426


In [5]:
ATTESTER_ADDR = "0xf6872d315cc2e1aff6abae5dd814fd54755fe97c"

df_project_metadata = pd.DataFrame(project_attestation_data)
df_project_metadata = df_project_metadata[df_project_metadata.attester.str.lower() == ATTESTER_ADDR]
df_project_metadata.sort_values(by="timeCreated", ascending=False, inplace=True)
df_project_metadata.drop_duplicates(subset="projectRefUID", keep="first", inplace=True)

PROJECTS = list(df_project_metadata['projectRefUID'].unique())
print("Projects:", len(PROJECTS))

Projects: 3040


In [6]:
df_applications = pd.DataFrame(app_attestation_data)
df_applications = df_applications[df_applications.attester.str.lower() == ATTESTER_ADDR]
df_applications['round'] = df_applications['round'].apply(lambda x: str(x))
df_applications.sort_values(by="timeCreated", ascending=False, inplace=True)
df_applications.drop_duplicates(subset="metadataSnapshotRefUID", keep="first", inplace=True)

latest_metadata = list(df_project_metadata.id.unique())
df_applications = df_applications[df_applications.metadataSnapshotRefUID.isin(latest_metadata)]
print("Applications:", len(df_applications))

Applications: 143


# Clean / filter attestation data

In [7]:
relevant_badgeholder_attestations = []
for a in badgeholder_attestation_data:
    if a['attester'] != '0xE4553b743E74dA3424Ac51f8C1E586fd43aE226F':
        continue
    if a['voterType'] == 'Guest':
        continue
    relevant_badgeholder_attestations.append({
        'id': a['id'].lower(),
        'attester': a['attester'].lower(),
        'recipient': a['recipient'].lower(),
        'timeCreated': datetime.datetime.utcfromtimestamp(a['timeCreated'])
    })
print("Relevant badgeholder attestations:", len(relevant_badgeholder_attestations))  
BADGEHOLDERS = set([a['recipient'].lower() for a in relevant_badgeholder_attestations])

Relevant badgeholder attestations: 200


In [8]:
mapped_roles = {
    'ACC (Second half)': 'Anticapture Commission',
    'Anticapture Commission Member': 'Anticapture Commission',
    'Badgeholder Reviewer': None,
    'Code of Conduct Council': 'Code of Conduct',
    'Code of Conduct Council Lead': 'Code of Conduct',
    'Code of Conduct Council Member': 'Code of Conduct',
    'Developer Advisory Board Lead': 'Developer Advisory Board',
    'Developer Advisory Board Member': 'Developer Advisory Board',
    'Optimism Grants Council Lead': 'Grants Council',
    'Optimism Grants Council Member': 'Grants Council',
    'Security Council Lead': 'Security Council',
    'Security Council Member': 'Security Council',
}
relevant_gov_council_attestations = []
for a in gov_council_attestation_data:
    if a['attester'] != '0xE4553b743E74dA3424Ac51f8C1E586fd43aE226F':
        continue
    if a['govSeason'] not in ['5', '6']:
        continue
    gov_role = mapped_roles.get(a['govRole'], 'unknown')
    if not gov_role:
        continue
    if gov_role == 'unknown':
        print("Unknown role:", a['govRole'])
        continue
    relevant_gov_council_attestations.append({
        'id': a['id'].lower(),
        'attester': a['attester'].lower(),
        'recipient': a['recipient'].lower(),
        'govRole': gov_role,
        'timeCreated': datetime.datetime.utcfromtimestamp(a['timeCreated'])
    })
print("Relevant gov council attestations:", len(relevant_gov_council_attestations))   
COUNCILS = list(pd.DataFrame(relevant_gov_council_attestations)['govRole'].unique())
COUNCILS

Relevant gov council attestations: 60


['Grants Council',
 'Anticapture Commission',
 'Developer Advisory Board',
 'Code of Conduct',
 'Security Council']

# Determine approved attestations for RF6

In [9]:
ATTESTERS = {}
for addr in BADGEHOLDERS:
    if addr not in ATTESTERS:
        ATTESTERS.update({
            addr: {
                'is_citizen': True,
                'is_top_delegate': False,
                'governance_membership': set()                
            }
        })
for addr in DELEGATES:
    if addr not in ATTESTERS:
        ATTESTERS.update({
            addr: {
                'is_citizen': False,
                'is_top_delegate': True,
                'governance_membership': set()
            }
        })
    else:
        ATTESTERS[addr]['is_top_delegate'] = True

for a in relevant_gov_council_attestations:
    addr = a['recipient'].lower()
    if addr in ATTESTERS:
        ATTESTERS[addr]['governance_membership'].add(a['govRole'])  
        
for addr in ATTESTERS:
    ATTESTERS[addr]['governance_membership'] = list(ATTESTERS[addr]['governance_membership'])    

print(len(ATTESTERS.keys()))

188


# Create attestations dataframe

In [10]:
def convert_farcaster(farcaster_id):
    return int(farcaster_id['hex'], 16) if farcaster_id else None

In [11]:
mgl_attester = '0x7484aABFef9f39464F332e632047983b67571C0a'

reviewed_attestations = []
relevant_impact_attestations = []
for attestation in impact_attestation_data:

    a = attestation.copy()
    fid = convert_farcaster(a['farcasterID'])
    a['farcasterID'] = fid
    a['timeCreated'] = datetime.datetime.utcfromtimestamp(a['timeCreated'])
    
    if a['attester'] != mgl_attester:
        a.update({'status': 'unknown_signer'})
        reviewed_attestations.append(a)
        continue
        
    if a['projectRegUID'] not in PROJECTS:
        a.update({'status': 'unknown_project'})
        reviewed_attestations.append(a)
        continue

    user_addresses = FARCASTER.get(fid, set())
    if not len(user_addresses):
        a.update({'status': 'farcaster_has_no_eoa'})
        reviewed_attestations.append(a)
        continue
    
    linked_addresses = user_addresses.intersection(set(ATTESTERS.keys()))
    if not linked_addresses:
        a.update({'status': 'farcaster_not_gov_participant'})
        reviewed_attestations.append(a)
        continue

    if len(linked_addresses) > 1:
        print(fid)
    linked_address = linked_addresses.pop()
        
    qs = a.get('metadata', {}).get('impactAttestations', [])
    if len(qs) < 2:
        a.update({'status': 'missing_metadata'})
        reviewed_attestations.append(a)
        continue
    
    nps = pmf = None
    for q in qs:
        val = q.get('value')
        if not val:
            continue
        if q['name'] == 'Likely to Recommend':
            nps = val
        elif q['name'] == 'Feeling if didnt exist':
            pmf = val
    
    if not (nps or pmf):
        a.update({'status': 'missing_scores'})
        reviewed_attestations.append(a)
        continue

    a.update({'status': 'OK'})
    reviewed_attestations.append(a)
    relevant_impact_attestations.append({
        'id': a['id'].lower(),
        'attester': a['attester'].lower(),
        'recipient': a['recipient'].lower(),
        'linkedAddress': linked_address,
        'farcasterID': fid,
        'projectRegUID': a['projectRegUID'].lower(),
        'timeCreated': a['timeCreated'],
        'issuer': a['issuer'],
        'metadataurl': a['metadataurl'],
        'nps_score': pd.to_numeric(nps),
        'pmf_score': pd.to_numeric(pmf)
    })
    
df_normalized_attestations = pd.DataFrame(reviewed_attestations)
df_normalized_attestations.to_csv("data/normalized_attestations.csv")    
print("Reviewed impact attestations:", len(df_normalized_attestations))
df_normalized_attestations['status'].value_counts()

Reviewed impact attestations: 426


status
farcaster_not_gov_participant    97
unknown_project                  93
OK                               85
farcaster_has_no_eoa             75
unknown_signer                   67
missing_metadata                  9
Name: count, dtype: int64

# Implement metrics on relevant attestations

In [12]:
df = pd.DataFrame(relevant_impact_attestations)
df['is_citizen'] = df.linkedAddress.apply(lambda x: ATTESTERS[x]['is_citizen'])
df['is_top_delegate'] = df.linkedAddress.apply(lambda x: ATTESTERS[x]['is_top_delegate'])
df['governance_membership'] = df.linkedAddress.apply(lambda x: ATTESTERS[x]['governance_membership'])
print("Relevant impact attestations:", len(df))
df['projectRegUID'].value_counts()

Relevant impact attestations: 85


projectRegUID
0xe2247097961708233325f253d676ba2f9d8011147e1bb08e0e50d55894138dc7    6
0x98877a3c5f3d5eee496386ae93a23b17f0f51b70b3041b3c8226f98fbeca09ec    5
0x51504348243d8e8fa59cf2ba983d9255373e5eb23fe22a0d18c0d36028561ace    4
0xdd197ad6aa36b5d3753c1710960b40f1e23e941b6967444afe36860c3f2e288c    4
0x6b68c75e9dcd10c64f2b440e47589eb9ef29e17352055816e4f974401eab2358    4
0xfb52bbc12284743f2bf1ad88e2b1b5b3b08d88fa1a7414eefd256ee2ad4e6c73    4
0x5e6c436e48e56d6d9622ba5d0be0035c314e2b29d2afc8f5f1ee8ac75cd42532    4
0x3ecbf66c1b21342b26acb0f7d4c6b8bbbef168ffafff3f425de3503fd8dfa535    3
0x1e08897960cdb18df87f253f4e0f047b2edba189ea7403e5448ed2ba443f0013    3
0xbb1e4aadda991e93d4ba4630582f222b2c57c0b0d729e29247cf2a3873936808    3
0x4d7ac580e10414ef62d024b14ad94ebd1434efacd1fd3f453560b2916f2606d2    3
0xa08c7ab7824d147d6cb87ffb4e86cec454a030d48b8c395bad6d64456be3d911    3
0x6aa80764b6082947c3b2de86fe12804eb475b0afb719de50b9eed60b86f20535    3
0xa8c946244b7b2b556c9a5bba875d7d86f13de7758e6c16c5

In [13]:
metrics = calculate_metrics(df)

with open("data/attestation_metrics.json", "w") as f:
    json.dump(metrics,f,indent=2)

# Create dummy attestations

In [30]:
import random
import numpy as np
import secrets

class CustomJSONizer(json.JSONEncoder):
    def default(self, obj):
        return super().encode(bool(obj)) \
            if isinstance(obj, np.bool_) \
            else super().default(obj)

In [31]:
NUM_PROJECTS = 30
NUM_ATTESTERS = 100
NUM_ATTESTATIONS = 1000
PMF_SCALE = 3
NPS_SCALE = 10


projects = {
    p: random.random()
    for p in random.choices(PROJECTS, k=NUM_PROJECTS)
}

attesters = {
    addr:ATTESTERS[addr]
    for addr in random.choices(list(ATTESTERS.keys()), k=NUM_ATTESTERS)
}

dummy_attestations = []
for i in range(NUM_ATTESTATIONS):
    random_project = random.choice(list(projects.keys()))
    random_attester = random.choice(list(attesters.keys()))
    project_impact = projects.get(random_project)
    pmf_score = random.randint(int(project_impact*PMF_SCALE), PMF_SCALE)
    nps_score = random.randint(int(project_impact*NPS_SCALE), NPS_SCALE)
    dummy_attestations.append({
        'id': f"0x{secrets.token_hex(32)}",
        'attester': random_attester,
        'projectRegUID': random_project,
        'pmf_score': max(1,pmf_score),
        'nps_score': max(1,nps_score),
        **attesters[random_attester]
    })

In [32]:
df_dummy = pd.DataFrame(dummy_attestations)
dummy_metrics = calculate_metrics(df_dummy)

In [33]:
with open("data/dummy_metrics_v2.json", "w") as f:
    json.dump(dummy_metrics, f, indent=2, cls=CustomJSONizer)